In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [7]:
datapath = "data/"
all_embeddings = []
all_parametric = []
for i in trange(2048):
    dir = datapath + f"batch_{i}/"
    embeddings = pd.read_csv(dir + "embeddings.csv", index_col=0)
    parametric = pd.read_csv(dir + "parametric.csv", index_col=0)
    all_embeddings.append(embeddings)
    all_parametric.append(parametric)
all_embeddings = pd.concat(all_embeddings)
all_parametric = pd.concat(all_parametric)


100%|██████████| 2048/2048 [03:35<00:00,  9.52it/s]


In [6]:
all_embeddings

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,0,0.106251,0.258809,0.217097,0.052001,0.067766,-0.596467,0.020813,0.508006,0.803247,...,0.060043,0.234398,-0.063276,0.037449,-0.327099,-0.654285,-0.061786,0.441500,0.296609,-0.443207
1,1,0.010169,0.182377,0.143281,0.064662,0.079366,-0.568401,0.050981,0.455648,0.862232,...,0.016913,0.190918,0.022237,0.108317,-0.310745,-0.472191,-0.076475,0.258456,0.258676,-0.456437
2,12,0.033307,0.097303,0.197690,-0.012557,-0.141809,-0.445748,0.010131,0.358836,0.593737,...,0.080775,0.318575,0.091919,0.309961,-0.310382,-0.344101,0.029656,0.151467,0.189677,-0.167317
3,17,0.029129,0.014132,0.345716,0.033893,0.102281,-0.331283,0.049822,0.334491,0.513428,...,0.285765,0.357120,-0.123767,0.437610,-0.243112,-0.183980,0.014640,0.350762,0.105127,-0.182382
4,18,0.208439,0.010890,0.187300,0.022743,-0.024630,-0.616475,0.124953,0.315829,0.999170,...,0.074361,0.281442,-0.106480,0.227237,-0.282826,-0.249534,-0.117560,0.282691,0.206000,-0.140912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,8147,0.044186,0.100299,0.135002,-0.020155,-0.058140,-0.486102,-0.007282,0.437817,0.642198,...,0.119772,0.202130,-0.003237,0.243974,-0.191658,-0.342585,0.202473,0.302443,0.228328,-0.233146
688,8153,0.035728,0.173219,0.202191,0.189068,0.026263,-0.585359,0.001681,0.397620,0.926002,...,-0.025365,0.201999,0.055369,0.374553,-0.279251,-0.429589,-0.116543,0.098589,0.215639,-0.236826
689,8157,-0.119564,0.124329,0.235055,-0.044418,-0.351188,-0.300287,0.010032,0.395619,0.596754,...,0.047691,0.349019,0.003443,0.242602,-0.306653,-0.247859,0.050787,0.214852,0.274497,-0.408027
690,8174,0.045260,0.185526,0.133300,-0.027407,-0.194929,-0.390436,-0.095164,0.376591,0.547237,...,0.178840,0.096593,0.135894,0.092209,-0.271243,-0.440581,-0.016183,0.155191,0.265883,-0.176978


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Residual Block class
class ResidualBlock(nn.Module):
    def __init__(self, input_size, layer_size, num_layers):
        super(ResidualBlock, self).__init__()
        self.layers = self._make_layers(input_size, layer_size, num_layers)

    def _make_layers(self, input_size, layer_size, num_layers):
        layers = []
        layers.append(nn.Linear(input_size, layer_size))
        layers.append(nn.ReLU(inplace=True))
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(layer_size, layer_size))
            layers.append(nn.ReLU(inplace=True))
        return nn.Sequential(*layers)

    def forward(self, x):
        residual = x
        out = self.layers(x)
        out += residual
        return out


# Residual Network class
class ResidualNetwork(nn.Module):
    def __init__(self, input_size, output_size, layer_size, layers_per_block, num_blocks):
        super(ResidualNetwork, self).__init__()
        self.initial_layer = nn.Linear(input_size, layer_size)
        self.blocks = self._make_blocks(layer_size, layers_per_block, num_blocks)
        self.final_layer = nn.Linear(layer_size, output_size)

    def _make_blocks(self, layer_size, layers_per_block, num_blocks):
        blocks = []
        for _ in range(num_blocks):
            blocks.append(ResidualBlock(layer_size, layer_size, layers_per_block))
        return nn.Sequential(*blocks)

    def forward(self, x):
        out = self.initial_layer(x)
        out = self.blocks(out)
        out = self.final_layer(out)
        return out


# Example usage
input_size = 100
output_size = 10
layer_size = 256
layers_per_block = 2
num_blocks = 3

model = ResidualNetwork(input_size, output_size, layer_size, layers_per_block, num_blocks)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Sample input and target
input_data = torch.randn(1, input_size)
target = torch.empty(1, dtype=torch.long).random_(output_size)

# Training loop example
for epoch in range(10):
    optimizer.zero_grad()
    output = model(input_data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')